In [1]:
from rasa.nlu.model import Interpreter

In [2]:
model = Interpreter.load("nlu")

2021-07-10 23:24:30.508447: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-10 23:24:30.525794: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9a4057b360 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-07-10 23:24:30.525807: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [13]:
model.model_metadata.model_dir

'nlu'

In [3]:
intents = ["greet", "goodbye", "bot_challenge", "password_reset", "inform", 
"thank", "help", "problem_email", "open_incident", "incident_status", 
"out_of_scope", "restart", "affirm", "deny", "trigger_handoff", 
"human_handoff", "handoff"]

In [4]:
def predict_proba(model, text_list):
    results = []
    
    labels = ["greet", "goodbye", "bot_challenge", "password_reset", "inform", 
               "thank", "help", "problem_email", "open_incident", "incident_status", 
               "out_of_scope", "restart", "affirm", "deny", "trigger_handoff", 
               "human_handoff", "handoff"]
    
    for text in text_list:
        probs = []
        output = model.parse(text, only_output_properties=False)
        confs = {d['name']:d['confidence'] for d in output['intent_ranking'] if "id" in d}
        for label in labels:
            prob = 0.0
            if label in confs:
                prob = confs[label]
            probs.append(prob)
        results.append(probs)
        
    return results

# Unbox

In [5]:
import unboxapi
client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### upload model

In [6]:
from unboxapi.models import ModelType

model_path = client.add_model(
    function=predict_proba, 
    model=model,
    model_type=ModelType.rasa,
    class_names=intents,
    name='rasa model',
    description='this is my rasa model'
)

/Users/unbox/Desktop/unboxapi-python-client/examples/local/temp6
/Users/unbox/Desktop/unboxapi-python-client/examples/local/temp6/TemplateModel/nlu


# Test Bento

In [7]:
import bentoml

In [8]:
bento_model = bentoml.load(model_path)

[2021-07-10 23:27:22,601] WARNING - Module `template_model` already loaded, using existing imported module.


In [9]:
bento_model.predict({"text": ["hello"]})

{'text': ['hello']}


[[0.9999524354934692,
  0.0,
  9.283241979574086e-07,
  1.721418101396921e-07,
  3.308567784188199e-07,
  1.750023784552468e-06,
  1.4346930583997164e-05,
  3.443972673267126e-06,
  1.5785906271048589e-06,
  0.0,
  2.3841097984700355e-08,
  0.0,
  2.5017172447405756e-05,
  0.0,
  0.0,
  0.0,
  0.0]]

In [ ]:
!bentoml run TemplateModel:latest predict --input '{"text": ["how are you"]}'

In [12]:
!bentoml serve TemplateModel:latest

[2021-07-10 23:28:55,443] INFO - Getting latest version TemplateModel:20210710231743_D4383B
[2021-07-10 23:28:55,452] INFO - Starting BentoML API proxy in development mode..
[2021-07-10 23:28:55,455] INFO - Starting BentoML API server in development mode..
[2021-07-10 23:28:55,623] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
2021-07-10 23:29:01.910419: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-10 2